# Current Idea (open to other ideas of course!):

## Micro-Regime Autoencoders (MRAE)

### Problem Statement

Traditional factor models can struggle capturing short-horizon "micro-regimes" in stock price behavior. The Goal of this project is to use a small autoencoder to learn regimes from 20-day *(I just picked a number we can change this obviously)* sliding windows of price/volume features, and then studiy:

- What the regimes look like
- How they transition from one to another
- Whether transitions have predictive power for near-term returns or volatitlity
- Potential comparissons to simple baselines (momentum, volatility)

### Optional Download (You shouldn't need to do this) Please use the link in the readme to get the dataset

In [ ]:
# uncomment to download data. 
# Not needed if you have the .parquet file in the data/historical folder

# !python qualifier\utils\download_stock_data.py

### Load Dataset from file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Set random seeds for reproducibility
np.random.seed(42)

# Data path
PARQUET_PATH = Path("data/historical/all_stocks_historical.parquet")

print(f"Loading data from: {PARQUET_PATH}")
df = pd.read_parquet(PARQUET_PATH)
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn dtypes:")
print(df.dtypes)

Loading data from: data\historical\all_stocks_historical.parquet

DataFrame shape: (89859344, 7)

Column dtypes:
ticker            object
date      datetime64[us]
open             float64
high             float64
low              float64
close            float64
volume           float64
dtype: object


In [2]:
# Standardize and prepare data
print("Available columns:", df.columns.tolist())

# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Sort by ticker and date
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

print("\n" + "=" * 60)
print("Data Summary:")
print("=" * 60)
print(f"Total rows: {len(df):,}")
print(f"Unique tickers: {df['ticker'].nunique():,}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Days covered: {(df['date'].max() - df['date'].min()).days:,} days")
print(f"Date column type: {df['date'].dtype}")

Available columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']

Data Summary:
Total rows: 89,859,344
Unique tickers: 7,144
Date range: 1962-01-02 to 2025-11-13
Days covered: 23,326 days
Date column type: datetime64[us]


In [4]:
# original data with nans
print("\n" + "=" * 60)
print("Original Data:")
print("=" * 60)
display(df.head())

# no nan 
print("\n" + "=" * 60)
print("valid (non-null) data:")
print("=" * 60)

# remove rows with nans
df_valid = df.dropna(subset=['close', 'volume']).head()
display(df_valid[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']])


Original Data:


,ticker,date,open,high,low,close,volume
0,A,1962-01-02,NaN,NaN,NaN,NaN,NaN
1,A,1962-01-03,NaN,NaN,NaN,NaN,NaN
2,A,1962-01-04,NaN,NaN,NaN,NaN,NaN
3,A,1962-01-05,NaN,NaN,NaN,NaN,NaN
4,A,1962-01-08,NaN,NaN,NaN,NaN,NaN



valid (non-null) data:


,ticker,date,open,high,low,close,volume
9539,A,1999-11-18,27.245214,29.939798,23.951837,26.347021,62546380.0
9540,A,1999-11-19,25.710805,25.748231,23.839568,24.176392,15234146.0
9541,A,1999-11-22,24.737756,26.347021,23.989261,26.347021,6577870.0
9542,A,1999-11-23,25.448832,26.122478,23.951841,23.951841,5975611.0
9543,A,1999-11-24,24.026691,25.112009,23.951841,24.588062,4843231.0
